In [1]:
import numpy as np
import scipy.io
import os
import pandas as pd
import math
from pathlib import Path
from mayavi import mlab
from pyquaternion import Quaternion
from IPython.core.debugger import set_trace

Python                    3.7
numpy                     1.17.2 
pyqt5                     5.13.1
mayavi                    4.7.1
https://docs.enthought.com/mayavi/mayavi/
vtk
scipy                     1.3.1 
pyquaternion              0.9.5 
http://kieranwynn.github.io/pyquaternion/

## Libaries

### Python
This was writen on python 3.7 (although 3.6 *should* work- although not tested) python 2 versions won't work due to the use of f strings

### mayavi
This is the 3d plotting libray used for rendering the plots. Mayvai will launch a qt window to display the plot- meaning that you will need an X serve session for the plots to load (would not recomend if you are using docker- there are ways of geting jupyter to show them inline, howwever I couldn't get it working!

#### mayavi install
https://docs.enthought.com/mayavi/mayavi/installation.html#installing-with-conda-forge

    conda install vtk
    conda install qyqt5
    
    conda install mayavi
    
### pyquaternion
http://kieranwynn.github.io/pyquaternion/

## matlab_loader

This makes a data loader class for matlab files and also passes a filter to remove noise 

In [2]:
class matlab_loader:
    """ Loads .mat files from a direcotory 
            
            root_dir (str): path to the directory that containes files
            mat_file (str): name of file 
     
         returns:
             self.array (np.array)
             self.mat_file (Path object)
    """
    def __init__(self, root_dir, mat_file):  
        
        root_dir = Path(root_dir)
        mat_file = Path(mat_file)
        
        mat_obj = scipy.io.loadmat(root_dir/mat_file)
        obj= mat_obj.keys()
        obj = list(obj)
        array = mat_obj[obj[-1]]
        
        self.array = array
        self.mat_file = root_dir/mat_file
        
    def __repr__(self):
        return f'{self.mat_file}'

In [3]:
def xyz(arr, filter_level = 0.3):
    """Convert 3D voxel arry to xyz coordiates.
    
            arr (np.array): 3D voxel array    
            filter_level (int/float): sets the threshold level for 
            what is considered a voxel 
            
            returns: 
                np.array (n x 3)
    """
    
    # Everything above filter level is converted to 1
    arr = np.where(arr < filter_level, 0, 1)
    
    x, y, z = np.where(arr == 1)
    
    # converts the xyz so z is is *up* 
    x -= arr.shape[1]
    y -= arr.shape[0]
    x *= -1
    y *= -1
    xyz = np.array([x, y, z]).T
    return xyz

## PCA

In [4]:
def pca(xyz):
    """PCA on a xyz points array
            xyz(np.array): n x 3 array of xyz coordinates
    """
    
    #covaraince of xyz points
    cov = np.cov(xyz.T)
    
    #eiganvalues and vectors
    (eig_val, eig_vec) = np.linalg.eig(cov)

    mean_x, mean_y, mean_z = [np.mean(xyz[:,0]),np.mean(xyz[:,1]),np.mean(xyz[:,2])]

    return mean_x, mean_y, mean_z, eig_val, eig_vec

# Classes:

## Hyriachy

1. `foot_bone` : sets which bone; eg 'tibia' 
   
2. `n_ang` : defines how many angles the scan has, also stores the f1 and/or f2 position

3. `bone` : It calls the PCA on xyz of the bone and stores the output


## Bone

In [ ]:
class bone:
        def __init__(self, array, filter_level=0.3):
          
            if array is None:
                pass
            
            else:
                self.array = array
                self.filter_level = filter_level
                self.xyz = xyz(array, filter_level)

                mean_x, mean_y, mean_z, eig_val, eig_vec = pca(self.xyz) 

                self.vec = eig_vec
                self.PC1 = eig_vec[:,0]
                self.PC2 = eig_vec[:,1]
                self.PC3 = eig_vec[:,2]   

                self.mean = (mean_x, mean_y, mean_z)

### Foot bone

In [6]:
class foot_bone(bone):
    def __init__(self, name = 'UN-NAMED', **kwargs):
        
        for key, value in kwargs.items():
            setattr(self, key, value)
            
        self.name = name
        
    def __repr__(self):
        return self.name

### Indivual bone classes

In [7]:
class n_ang(foot_bone):
    def __init__(self, n_ang_name, f1 = None ,f2 = None):
                  
        self.f1 = bone(f1)
        self.f2 = bone(f2)
        self.n_name = n_ang_name
        
    def __repr__(self):
        return f'{self.n_name} ' 

# Ploting:

Creates some repeated blocks for functions

In [8]:
def voxel_points(bone, color):
    mlab.points3d(bone[:,0],
                  bone[:,1],
                  bone[:,2], 
                  mode="cube", color=color)
    
def voxel_PCAs(bone,pc):
    pc_color = [(1,0,0),(0,1,0), (0,0,1)]
    
    for n in range(1,4):
        mlab.quiver3d(0,0,0, 
                      getattr(bone,f'{pc}{n}')[0],
                      getattr(bone,f'{pc}{n}')[1], 
                      getattr(bone,f'{pc}{n}')[2], 
                      line_width =6, scale_factor= 100, color= pc_color[n-1])
        

## bone_plot

This plots an arbatory number of bones

eg: `bone_plot(tibia.phantom.f1)`

In [9]:
# NEED TO REFACTOR
def bone_plot(*args, user_colours = None, plot_PCA = True, plot_inv = False):
    """plots voxel array; can take n bones and plot PCA vectors"""

    # Sorting out colours
    colour_dict = {'yellow':(0.9,0.9,0),
                   'pastel_blue':(0.7,1,1),
                   'purple':(0.6,0,0.5),
                   'orange':(0.8,0.3,0),
                   'dark_blue':(0,0.3,0.7),}
    
    if user_colours is None:
        user_colours = colour_dict
    
    plot_colours = []
    
    for col in user_colours:
        x = colour_dict.get(col)
        plot_colours.append(x)
        
    
    for n, bone in enumerate(args):

        mlab.points3d(bone.xyz[:,0], 
                      bone.xyz[:,1], 
                      bone.xyz[:,2],
                     mode="cube",
                     color= plot_colours[n],
                     scale_factor=1)
        
        x,y,z = bone.mean

        #plot princible vectors
        u0,v0,w0 = bone.PC1 * 100
        u0_inv,v0_inv,w0_inv = bone.PC1 * 100 * -1

        u1,v1,w1 = bone.PC2 * 100
        u1_inv,v1_inv,w1_inv = bone.PC2 * 100 * -1

        u2,v2,w2 = bone.PC3 * 100
        u2_inv,v2_inv,w2_inv = bone.PC3 * 100 * -1

        print(f"{n}th bone PCA vectors: \n {bone.vec} \n ")
        
        
        if plot_PCA is True:
            mlab.quiver3d(x,y,z,u0,v0,w0,
                                 line_width =6,
                                 scale_factor=0.7,
                                 color= (1,0,0))
            mlab.quiver3d(x,y,z,u1,v1,w1,
                                 line_width =6,
                                 scale_factor= 0.5,
                                 color= (0,1,0))
            mlab.quiver3d(x,y,z,u2,v2,w2, 
                                 line_width =6,
                                 scale_factor=0.3,
                                 color=(0,0,1))


        #ploting the inverse of eigen vectors
        if plot_inv is True:
            mlab.quiver3d(x,y,z,u0_inv,v0_inv,w0_inv,
                                 line_width =6,
                                 scale_factor=0.7,
                                 color= (1,0,0))
            mlab.quiver3d(x,y,z,u1_inv,v1_inv,w1_inv,
                                 line_width =6,
                                 scale_factor=0.5,
                                 color= (0,1,0))
            mlab.quiver3d(x,y,z,u2_inv,v2_inv,w2_inv,
                                 line_width =6,
                                 scale_factor=0.3,
                                 color=(0,0,1))

    return mlab.show()

In [504]:
# Option user colours
# user_colours=['yellow','purple']

## Angels

In [10]:
def mag(v):
    """Finds magnitude of vector
        v (np.array): vector
    """
    return math.sqrt(np.dot(v,v))


def angle(v1, v2):
    """Finds angel between vectors"""
    try:
        x = math.acos(np.dot(v1, v2) / (mag(v1) * mag(v2)))
    
    except:
        x = 0
        
    return x

In [ ]:
# # write a file loader 

# file list = walk through dir
#     for file in file list:
#          file[:-4] = matlab_loader(file)
#          setattr(bone, file[:-7])

In [18]:
# REFACTOR
#have __iter__ so you can loop through PCAs
def df_angles(bone, ang3 = None, ang6 = None, ang10 = None, ang15 = None, ALL = None):
    
    df = pd.DataFrame()
    
    #Phant f1 vs Phant f2 
    df.loc[f'{bone} phantom f1: PC1' ,f'{bone} phantom f2: PC1'] = angle(bone.phantom.f1.PC1,bone.phantom.f2.PC1)
    df.loc[f'{bone} phantom f1: PC2' ,f'{bone} phantom f2: PC2'] = angle(bone.phantom.f1.PC2,bone.phantom.f2.PC2)
    df.loc[f'{bone} phantom f1: PC3' ,f'{bone} phantom f2: PC3'] = angle(bone.phantom.f1.PC3,bone.phantom.f2.PC3)
 
    if ang3 or ALL is True:
        #Phant f2 vs ang_3 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang3 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang3.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang3 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang3.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang3 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang3.f2.PC3)

    if ang6 or ALL is True:
        #Phant f2 vs ang_6 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang6 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang6.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang6 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang6.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang6 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang6.f2.PC3)
    
    if ang10 or ALL is True:
        #Phant f2 vs ang_10 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang10 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang10.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang10 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang10.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang10 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang10.f2.PC3)
    
    if ang10 or ALL is True:
        #Phant f2 vs ang_15 f2
        df.loc[f'{bone} phantom f1: PC1' ,f'{bone} ang10 f2: PC1'] = angle(bone.phantom.f1.PC1,bone.ang15.f2.PC1)
        df.loc[f'{bone} phantom f1: PC2' ,f'{bone} ang10 f2: PC2'] = angle(bone.phantom.f1.PC2,bone.ang15.f2.PC2)
        df.loc[f'{bone} phantom f1: PC3' ,f'{bone} ang10 f2: PC3'] = angle(bone.phantom.f1.PC3,bone.ang15.f2.PC3)
    
    return df

# Rotation

## Step 1: Center the 2 means

Moves the f1 bone to the f2 position; 

Creates new atribute `bone.f1.tfm_xyz` and `bone.f2.tfm_xyz`

In [22]:
def voxel_center(bone):
    #moves f1 onto f2
    tfm =  np.asarray(bone.f1.mean) - np.asarray(bone.f2.mean)
    
    #changing bone matrix coords f1
    bone.f1.tfm_xyz = bone.f1.xyz + tfm
    
    #sets mean to origin
    for n, i in enumerate(bone.f1.mean):
         bone.f1.tfm_xyz[:,n] -= bone.f1.mean[n] 
    
    #changing bone matrix coords f2
    bone.f2.tfm_xyz = bone.f2.xyz.astype(np.float64)
    #sets mean to origin
    for n, i in enumerate(bone.f2.mean):
         bone.f2.tfm_xyz[:,n] -= bone.f1.mean[n] 
            
    return bone

# Quatertions

## Rotaion around PCs

In [24]:
def roation(bone):
    
    # init tfm_PCn
    for n in range(1,4):
        setattr(bone.f1,f'tfm_PC{n}',getattr(bone.f1,f'PC{n}'))
        
    # for each pc rotate    
    for n in range(1,4):
    
        f1_PCn = getattr(bone.f1,f'tfm_PC{n}')
        f2_PCn = getattr(bone.f2,f'PC{n}')
            
        # angle between PCs
        ang =  angle(f1_PCn, f2_PCn)
        
        #cross product between PCs
        x,y,z = np.cross(f1_PCn, f2_PCn)

        r = Quaternion(axis = [x,y,z], angle = ang)
        
        # rotate voxelx
        #set_trace()
        bone.f1.tfm_xyz = np.apply_along_axis(r.rotate, 1, bone.f1.tfm_xyz)
        print(f'{n} {r}')
        
        #rotate PCs
        for n in range(1,4):
            setattr(bone.f1,
                    f'tfm_PC{n}',
                    r.rotate(getattr(bone.f1,f'tfm_PC{n}')))
                                            
    return bone

In [25]:
def rotation_plot(bone):
    
   #plot rotated f1
    voxel_points(bone.f1.tfm_xyz,color=(0,0.7,0))
    
    # plots orginal f2 
    voxel_points(bone.f2.tfm_xyz,color=(0.7,0,0))

    #f2 PCA 
    voxel_PCAs(bone.f2,'PC')
    
    #w/ rotion
    voxel_PCAs(bone.f1,'tfm_PC') 

    mlab.show()

# How to use:

### 1. Set the root directory for the matlab file loader

In [182]:
root_dir = Path('C:\\Users\luke\OneDrive - University College London\Marta\data')

### 2. Load the data that you want to use

In [184]:
# load data phantoms
tibia_phant_f2 = matlab_loader(root_dir, mat_file = 'phantom/phantom_tibia_f2.mat' )
tibia_phant_f1 = matlab_loader(root_dir, mat_file = 'phantom/phantom_tibia_f1.mat')

# load mulitpul angles
tibia_3_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/3 angles/tibia_f2.mat')
tibia_6_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/6 angles/tibia_f2.mat')
tibia_10_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/10 angles/tibia_f2.mat')
tibia_15_f2 = matlab_loader(root_dir, mat_file = 'fista_recons/15 angles/tibia_f2.mat')

### 3. Contstruct the bone classes

In [183]:
# set the foot bone
tibia = foot_bone(name = 'tibia')

# load f1/f2 phantom data
tibia.phantom = n_ang(f1=tibia_phant_f1.array,
                      f2=tibia_phant_f2.array,
                      n_ang_name= 'phantom')

# load f2 3 angel data
tibia.ang3 = n_ang(f2 = tibia_3_f2.array,
                   n_ang_name= 'angle 3')

### 4. Change xyz coordiates so `f1` is in `f2` position

In [23]:
voxel_center(tibia.phantom)

phantom 

### 5. Rotate bone using PCs

In [67]:
roation(talus.phantom)

1 +0.304 -0.635i -0.004j -0.710k
2 +0.892 +0.271i -0.268j -0.241k
3 +1.000 -0.000i -0.000j +0.000k


phantom 

### 6. Plotting the rotation

In [68]:
rotation_plot(talus.phantom)

# Testing

In [35]:
talus = foot_bone(name = 'talus')

talus.phantom = n_ang(f1=talus_phant_f1.array,
                      f2=talus_phant_f2.array,
                      n_ang_name= 'phantom')

In [171]:
%%time
tibia = foot_bone(name = 'tibia')

tibia.phantom = n_ang(f1=tibia_phant_f1.array,
                      f2=tibia_phant_f2.array,
                      n_ang_name= 'phantom')

tibia.ang3 = n_ang(f2 = tibia_3_f2.array,
                   n_ang_name= 'angle 3')
    

# ang6(f2 = tibia_6_f2.array),
# ang10(f2 = tibia_10_f2.array),
# ang15(f2 = tibia_15_f2.array)

Wall time: 1.06 s


In [172]:
tibia.phantom =voxel_center(tibia.phantom)

In [173]:
# edit data

In [174]:
x = Quaternion(axis = [0,1,0], angle = 2)

In [175]:
vox_obj = np.apply_along_axis(x.rotate, 1, tibia.phantom.f1.xyz)

In [176]:
mean_x, mean_y, mean_z, eig_val, eig_vec = pca(vox_obj)

# ...pc1

In [137]:
# # inv module 
# pc1_inv = talus.phantom.f2.PC1*-1
# ang =  angle(talus.phantom.f1.PC1 ,pc1_inv)
# x,y,z = np.cross(talus.phantom.f1.PC1,pc1_inv)

In [138]:
tibia.phantom.f1.PC1, tibia.phantom.f1.PC2, tibia.phantom.f1.PC3

(array([ 0.33443476,  0.03887988, -0.94161656]),
 array([0.92111808, 0.19774349, 0.33531924]),
 array([-0.19923572,  0.97948245, -0.03031934]))

In [139]:
#pca(talus.phantom.f1.tfm_xyz)

In [141]:
ang =  angle(tibia.phantom.f1.PC1, tibia.phantom.f2.PC1)
x,y,z = np.cross(tibia.phantom.f1.PC1, tibia.phantom.f2.PC1)

In [142]:
r1 = Quaternion(axis = [x,y,z], angle = ang);x,y,z

(-0.14904428122539115, -0.06990183523123583, -0.05582247149327461)

In [143]:
tibia.phantom.f1.tfm_xyz = np.apply_along_axis(r1.rotate, 1, tibia.phantom.f1.tfm_xyz)

In [144]:
pc1 = tibia.phantom.f1.PC1
pc2 = tibia.phantom.f1.PC2
pc3 = tibia.phantom.f1.PC3

In [145]:
pc1= r1.rotate(pc1)
pc2= r1.rotate(pc2)
pc3= r1.rotate(pc3)

# ...pc2

In [146]:
#pc2_inv = tibia.phantom.f2.PC2*-1

In [147]:
#ang = (angle(pc2_inv ,tibia.phantom.f1.PC2))

In [148]:
# _, _, _, _, eig_vec2 = pca(talus.phantom.f1.tfm_xyz); eig_vec2

In [149]:
ang =  angle(pc2, tibia.phantom.f2.PC2)

In [150]:
ang

1.7228105809752479

In [151]:
x,y,z = np.cross(pc2, tibia.phantom.f2.PC2)

In [152]:
r2 = Quaternion(axis = [x,y,z], angle = ang)

In [153]:
pc1 = r2.rotate(pc1)
pc2 = r2.rotate(pc2)
pc3 = r2.rotate(pc3)

In [154]:
tibia.phantom.f1.tfm_xyz = np.apply_along_axis(r2.rotate, 1, tibia.phantom.f1.tfm_xyz)

# ...pc3

In [155]:
#pc3_inv = tibia.phantom.f2.PC3*-1

In [156]:
#ang = (angle(pc3_inv ,tibia.phantom.f1.PC3))

In [157]:
ang =  angle(pc3, tibia.phantom.f2.PC3)

In [158]:
x,y,z= np.cross(pc3, tibia.phantom.f2.PC3) ;x,y,z

(-2.220446049250313e-16, 5.273559366969494e-16, -7.216449660063518e-16)

In [159]:
r3 = Quaternion(axis = [x,y,z], angle = ang)

In [160]:
pc1 = r3.rotate(pc1)
pc2 = r3.rotate(pc2)
pc3 = r3.rotate(pc3)

In [161]:
tibia.phantom.f1.tfm_xyz = np.apply_along_axis(r3.rotate, 1, tibia.phantom.f1.tfm_xyz)

In [177]:
# roatated 
mlab.points3d(vox_obj[:,0],
              vox_obj[:,1],
              vox_obj[:,2], 
              mode="cube", color=(0,0.7,0))

#f2
mlab.points3d(tibia.phantom.f1.xyz[:,0],
              tibia.phantom.f1.xyz[:,1],
              tibia.phantom.f1.xyz[:,2],
              mode="cube", color=(1,0,0))

#f2 PCA 
mlab.quiver3d(0,0,0,tibia.phantom.f1.PC1[0],tibia.phantom.f1.PC1[1],tibia.phantom.f1.PC1[2], line_width =6, scale_factor= 100, color= (1,0,0))
mlab.quiver3d(0,0,0,tibia.phantom.f1.PC2[0],tibia.phantom.f1.PC2[1],tibia.phantom.f1.PC2[2], line_width =6, scale_factor= 100, color= (0,0,1))
mlab.quiver3d(0,0,0,tibia.phantom.f1.PC3[0],tibia.phantom.f1.PC3[1],tibia.phantom.f1.PC3[2], line_width =6, scale_factor= 100, color= (0,1,0))

#w/ rotion
mlab.quiver3d(0,0,0,eig_vec[0,0],eig_vec[0,1],eig_vec[0,2], line_width =6, scale_factor= 100, color= (0.6,0,0))
mlab.quiver3d(0,0,0,eig_vec[1,0],eig_vec[1,1],eig_vec[1,2], line_width =6, scale_factor= 100, color= (0,0,1))
mlab.quiver3d(0,0,0,eig_vec[2,0],eig_vec[2,1],eig_vec[2,2], line_width =6, scale_factor= 100, color= (0,1,0))

mlab.show()